In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
# img_width, img_height = 224, 224
img_width, img_height = 500, 500

pretrained_model = "inceptionv3_500"


top_model_weights_path = '../model/bottleneck_fc_model_%s.h5'%pretrained_model
# train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
# validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
# test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
# nb_train_samples = 1462
# nb_validation_samples = 167
# nb_test_samples = 171
train_data_dir = '../data/Pandora18K_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
test_data_dir = '../data/Pandora18K_train_val_test_split/test'
nb_train_samples = 14313
nb_validation_samples = 1772
nb_test_samples = 1791

epochs = 50
batch_size = 64



In [4]:
datagen = ImageDataGenerator(rescale=1. / 255)
# build the VGG16 network
# model = applications.VGG16(include_top=False, weights='imagenet')
model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')

In [ ]:




generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, (nb_train_samples // batch_size)+1,verbose=1)








In [ ]:
np.save(open('../model/bottleneck_features_train_%s.npy'%pretrained_model, 'w'),
        bottleneck_features_train)

In [ ]:
generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, (nb_validation_samples // batch_size)+1,verbose=1)
np.save(open('../model/bottleneck_features_validation_%s.npy'%pretrained_model, 'w'),
        bottleneck_features_validation)




generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_test = model.predict_generator(
    generator, (nb_test_samples // batch_size)+1,verbose=1)
np.save(open('../model/bottleneck_features_test_%s.npy'%pretrained_model, 'w'),
        bottleneck_features_test)

In [5]:
from keras.utils.np_utils import to_categorical

train_data = np.load(open('../model/bottleneck_features_train_%s.npy'%pretrained_model))
#full
train_labels = np.array([0]*684+[1]*598+[2]*655+[3]*657+[4]*808+[5]*675+[6]*715+[7]*946+[8]*995+[9]*1021+[10]*803+[11]*816+[12]*566+[13]*959+[14]*842+[15]*831+[16]*849+[17]*893)

#small
#train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)

train_labels = to_categorical(train_labels, num_classes=18)

validation_data = np.load(open('../model/bottleneck_features_validation_%s.npy'%pretrained_model))
#full
validation_labels = np.array([0]*72+[1]*73+[2]*72+[3]*93+[4]*78+[5]*74+[6]*85+[7]*124+[8]*131+[9]*118+[10]*109+[11]*105+[12]*80+[13]*130+[14]*108+[15]*89+[16]*111+[17]*120)

#small
#validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
validation_labels = to_categorical(validation_labels, num_classes=18)

test_data = np.load(open('../model/bottleneck_features_test_%s.npy'%pretrained_model))
#full
test_labels = np.array([0]*91+[1]*60+[2]*75+[3]*82+[4]*104+[5]*83+[6]*95+[7]*121+[8]*131+[9]*123+[10]*103+[11]*117+[12]*65+[13]*123+[14]*121+[15]*112+[16]*89+[17]*96)

#small
#test_labels = np.array([0]*14+[1]*11+[2]*12+[3]*10+[4]*11+[5]*7+[6]*12+[7]*11+[8]*10+[9]*8+[10]*8+[11]*7+[12]*12+[13]*8+[14]*6+[15]*6+[16]*10+[17]*8)
test_labels = to_categorical(test_labels, num_classes=18)

In [6]:
import keras
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(18, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path,overwrite=True)

Train on 14313 samples, validate on 1772 samples
Epoch 1/50
14313/14313 [==============================] - 74s - loss: 2.8293 - acc: 0.1079 - val_loss: 2.6428 - val_acc: 0.1749
Epoch 2/50
14313/14313 [==============================] - 73s - loss: 2.6462 - acc: 0.1444 - val_loss: 2.3619 - val_acc: 0.2540
Epoch 3/50
14313/14313 [==============================] - 73s - loss: 2.5294 - acc: 0.1640 - val_loss: 2.2881 - val_acc: 0.2754
Epoch 4/50
14313/14313 [==============================] - 73s - loss: 2.4061 - acc: 0.1930 - val_loss: 2.1115 - val_acc: 0.3239
Epoch 5/50
14313/14313 [==============================] - 74s - loss: 2.3325 - acc: 0.2071 - val_loss: 2.0815 - val_acc: 0.3347
Epoch 6/50
14313/14313 [==============================] - 74s - loss: 2.2614 - acc: 0.2211 - val_loss: 2.0238 - val_acc: 0.3736
Epoch 7/50
14313/14313 [==============================] - 73s - loss: 2.2239 - acc: 0.2315 - val_loss: 2.0264 - val_acc: 0.3623
Epoch 8/50
14313/14313 [==============================]

NameError: name 'top_model_weights_path' is not defined

In [8]:
top_model_weights_path = '../model/bottleneck_fc_model_%s_val_acc_4791.h5'%pretrained_model
model.save_weights(top_model_weights_path,overwrite=True)

In [9]:
model.evaluate(test_data, test_labels, batch_size=32, verbose=1, sample_weight=None)

1791/1791 [==============================] - 6s     


[1.6940838059255492, 0.48297040672824076]